# Semestrální práce KMA/NA

Jan Půlpán

## Úkoly 1.-3.

In [1]:
import numpy as np
import numpy.linalg as la
import scipy.linalg as sla

np.set_printoptions(precision=4, suppress=True)

### 1. Ortogonální podobnostní transformace.

Převedeme náhodnou čtvercovou matici $A \in \mathbb{R}^{n \times n}$ pomocí ortogonálních podobnostních transformací na matici $H$ v horním Hessenbergerově tvaru. Bude tedy platit $$QAQ^T = H.$$ Ke konstrukci matice matice $Q$ použijeme Houselholderovy reflexe.

In [2]:
def hessenberg(A):
    m, n = A.shape
    assert m == n, 'Not a square matrix.'
    
    Q = np.eye(m)
    H = A.copy()

    for k in range(n-2):
        #k-tý sloupec, od k+1 řádku
        v = np.copy(H[k+1:, k]) 
        # vypočítáme householderův vektor
        v[0] = v[0] + np.sign(v[0])*la.norm(v) 
        v = (v / la.norm(v)).reshape(-1,1)
        
        # nasobeni zleva
        H[k+1:, k:] = H[k+1:, k:] - 2 * v @ (v.T @ H[k+1:, k:])
        # nasobeni zprava 
        H[:, k+1:] = H[:, k+1:] - 2 * (H[:, k+1:] @ v) @ v.T 
                                             
        Q[k+1:] = Q[k+1:] - 2 * v @ (v.T @ Q[k+1:])
        
    return Q, H

Transformaci otestujeme na náhodné matici $A \in \mathbb{R}^{6 \times 6}$ s prvky $a_{ij} \in [-100, 100]$.

In [3]:
A = np.random.uniform(-100,100,(6,6))
print(A)

[[  0.6589 -92.4675  64.7778 -35.6905 -99.2047  59.7031]
 [-31.1709  46.4348  28.0304  35.6469  31.246   39.4784]
 [ 91.7574 -54.8067 -76.0436  30.9586  98.0911  -7.6056]
 [-51.8249 -71.5659 -83.6854 -12.0011 -61.9425   2.5674]
 [-39.8482   2.6921  76.2762 -91.4997  53.5056 -80.2281]
 [-76.3048 -56.3507  58.7068  43.5202  67.7773  89.1966]]


In [4]:
%%time
Q, H = hessenberg(A)
print(H)

[[   0.6589   72.1601  -77.2439   -2.7994  -26.6018  124.6478]
 [ 139.5964  -47.1881   54.1157   46.7142   33.0619  -27.0602]
 [  -0.      135.5714   27.3088   34.1713  -49.6992  -81.7634]
 [   0.        0.     -119.9666   39.6462   -0.8985  -40.1836]
 [   0.        0.        0.      153.3986   20.3314   53.6513]
 [   0.        0.        0.       -0.       10.648    60.994 ]]
CPU times: user 1.25 ms, sys: 709 µs, total: 1.96 ms
Wall time: 1.26 ms


Ověříme zda platí $QAQ^T = H$.

In [5]:
np.allclose(Q@A@Q.T, H, rtol=1e-10, atol=1e-12)

True

A pro kontrolu ještě porovnáme námi vygenerovanou matici $H$ s výstupem funkce `scipy.linalg.hessenberg(A)`.

In [6]:
np.allclose(sla.hessenberg(A),H)

True

### 2. Ortogonální podobnostní transformace symetrické matice.

Jaký bude výsledek, pokud použijeme transformaci z 1. úlohy na symetrickou matici $A$? Matice tranformací v jednotlivých iteracích mají následující tvar
$$ Q_i = \begin{bmatrix}
I_i & 0\\
0 & \tilde{Q_i}
\end{bmatrix}, \quad i \in \{1,..,n-2\}$$
kde $I_i$ je jednotková matice velikosti $i \times i$ a $\tilde{Q_i} \in \mathbb{R}^{(n-i) \times (n-i)}$ je matice Householderovy reflexe.

Při násobení matice $A$ maticí $Q_i$ zleva nuluje tato operace prvky v $i$-tém sloupci od $(i+2)$-tého řádku. Pokud pak matici $A$ vynásobíme maticí $Q_i^{-1} = Q_i^T$ zprava, provede se stejná operace v $i$-tém řádku od $(i+2)$-tého sloupce. Protože je matice $A$ symetrická, i v tomto případě se tak prvky vynulují. Postupně tak vznikne třídiagonální symetrická matice $H$.

Můžeme také psát $$H = QAQ^T = QA^TQ^T = (QAQ^T)^T = H^T.$$ Takže i matice $H$ je symetrická. A vzhledem k tomu, že je zároveň Hessenbergerova, musí být 3-diagonální.

In [7]:
a = np.random.uniform(-100,100,(6,6))
A = np.tril(a) + np.tril(a, -1).T
print(A)

[[-25.3719 -17.4823  74.0511 -78.8255   4.8412 -46.095 ]
 [-17.4823 -40.9976  -8.1827  59.4614  84.7024 -59.4907]
 [ 74.0511  -8.1827 -53.3186 -80.9217  18.4054 -31.2824]
 [-78.8255  59.4614 -80.9217 -61.2163 -36.533   20.561 ]
 [  4.8412  84.7024  18.4054 -36.533  -16.969   88.5181]
 [-46.095  -59.4907 -31.2824  20.561   88.5181 -83.1777]]


In [8]:
%%time
Q, H = hessenberg(A)
print(H)

[[ -25.3719  118.9573   -0.       -0.       -0.        0.    ]
 [ 118.9573   36.8665  -32.0807    0.        0.       -0.    ]
 [   0.      -32.0807 -111.156   107.1776   -0.       -0.    ]
 [   0.       -0.      107.1776  -91.9001  -59.0604    0.    ]
 [   0.        0.        0.      -59.0604 -116.5548  -72.5967]
 [   0.       -0.        0.        0.      -72.5967   27.0651]]
CPU times: user 859 µs, sys: 203 µs, total: 1.06 ms
Wall time: 930 µs


### 3. Ortogonální transformace

Opět transformujeme náhodnou čtvercovou matici $A \in \mathbb{R}^{n \times n}$ pomocí ortogonálních transformací zprava a zleva tentokrát na matici $B$ v horním bidiagonálním tvaru. Bude tedy platit $$U^TAV = B.$$ Ke konstrukci matic $U, V$ použijeme Houselholderovy reflexe .

In [9]:
def bidiagonal(A):
    m, n = A.shape
    assert m == n, 'Not a square matrix.'

    U = np.eye(m)
    V = np.eye(m)
    B = A.copy()

    for k in range(n-1):
        #k-tý sloupec, od k řádku
        v = np.copy(B[k:, k]) 
        # vypočítáme householderův vektor
        v[0] = v[0] + np.sign(v[0])*la.norm(v)
        v = (v / la.norm(v)).reshape(-1,1)
        
        #násobení zleva
        B[k:, k:] = B[k:, k:] - 2 * v @ (v.T @ B[k:, k:])     
        U[k:] = U[k:] - 2 * v @ (v.T @ U[k:])
        
        #k-tý řádek, od k+1 sloupce
        v = np.copy(B[k, k+1:]) 
        # vypočítáme householderův vektor
        v[0] = v[0] + np.sign(v[0])*la.norm(v)
        v = (v / la.norm(v)).reshape(-1,1)

        #násobení zprava
        B[:, k+1:] = B[:, k+1:] - 2 * (B[:, k+1:] @ v) @ v.T                                             
        V[k+1:] = V[k+1:] - 2 * v @ (v.T @ V[k+1:])
        
    return U.T, V.T, B

Funkci vyzkoušíme na náhodné čtvercové matici $A \in \mathbb{R}^{6 \times 6}$.

In [10]:
A = np.random.uniform(-100,100,(6,6))
U, V, B = bidiagonal(A)

print(B)

[[ 131.4899  163.3479   -0.       -0.       -0.       -0.    ]
 [  -0.      -74.0255   35.189     0.        0.        0.    ]
 [  -0.        0.     -114.0575 -150.1632   -0.        0.    ]
 [  -0.       -0.       -0.     -107.3879  -77.2931   -0.    ]
 [   0.        0.        0.        0.     -173.3919   21.2304]
 [   0.        0.        0.        0.        0.       84.3437]]


Nakonec otestujeme platnost $U^TAV = B.$

In [11]:
np.allclose(U.T@A@V, B)

True